# Open Street Map data

## Project Overview
To choose any area of the world in https://www.openstreetmap.org and use data wrangling techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for that part of the world. Finally, use SQL as the data schema to complete your project by storing, querying and aggregating the data.

### OSM Dataset
The area selected for this project is Dublin city in Ireland. 

* Location: Dublin, Ireland
* [OpenStreetMap URL](https://www.openstreetmap.org/export#map=11/53.3549/-6.2512)
* [MapZen URL](https://mapzen.com/data/metro-extracts/metro/dublin_ireland/)

## Data Overview

Let's get an idea of what the top level tags are. Since the file is quite large instead of reading it in to memory as an XML tree I'll use iterative parsing to process the map file and find out not only what tags are there, but also how many, to get the feeling on how much of which data you can expect to have in the map.

In [2]:
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint

OSMFILE = "dublin_ireland.osm"

def count_tags(filename):
# return a dictionary with the tag name as the key and number of times 
# this tag can be encountered in the map as value. 
    tags = {}
    # iteratively parse the file
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags: 
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

pprint.pprint(count_tags(OSMFILE))

{'bounds': 1,
 'member': 91716,
 'nd': 2028541,
 'node': 1469711,
 'osm': 1,
 'relation': 4989,
 'tag': 1060747,
 'way': 269763}


The file is 369.4 MB and there are nearly 5,000,000 top level tags. 

### Nodes

A node is one of the core elements in the OpenStreetMap data model. It consists of a single point in space defined by its latitude, longitude and node id. A third, optional dimension (altitude) can also be included. A node can also be defined as part of a particular layer=* or level=*, where distinct features pass over or under one another; say, at a bridge.
Nodes can be used to define standalone point features, but are more often used to define the shape or "path" of a way

### Ways

A way is an ordered list of nodes which normally also has at least one tag or is included within a Relation.  way can be open or closed. A closed way is one whose last node on the way is also the first on that way. A closed way may be interpreted either as a closed polyline, or an area, or both. An open way is way describing a linear feature which does not share a first and last node. Many roads, streams and railway lines are open ways.

Relations are used to model logical (and usually local) or geographic relationships between objects. They are not designed to hold loosely associated but widely spread items.

### Check fot potential problems in tags

Let's explore the data a bit more. Before I process the data and add it into a database, I'll check the
"k" value for each "tag" tag and see if there are any potential problems.
We would like to change the data model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}. So, we have to see if we have such tags, and if we have any tags with
problematic characters.

In [3]:
import xml.etree.cElementTree as ET
import re

"""
we want to get a count of each of four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
"""
#Regex patterns
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# check the "k" value for each "<tag>"
def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            if lower.search(tag.attrib['k']):
                keys['lower'] += 1
            elif lower_colon.search(tag.attrib['k']):
                keys['lower_colon'] += 1
            elif problemchars.search(tag.attrib['k']):
                print(tag.attrib)
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, 
            "lower_colon": 0, 
            "problemchars": 0, 
            "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys 

process_map("dublin_ireland.osm")

{'lower': 673577, 'lower_colon': 342839, 'other': 44331, 'problemchars': 0}

### Number of contributors

Im intersted in finding out how many unique users have contributed to this map. 

In [4]:
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import re

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            continue    
    return users


def test():
    users = process_map("dublin_ireland.osm")
    print('Number of unique contributors:', len(users))

test()

Number of unique contributors: 1602


## Data Auditing

We will take two steps to audit the data:
1. Audit the OSMFILE and create a variable, 'mapping', to reflect the changes needed to fix the unexpected street types to the appropriate ones in the expected list. Only add mappings only for the actual problems found in this OSMFILE, not a generalized solution, since that may and will depend on the particular area being auditing.
2. Write a function to actually fix the street name. The function takes a string with street name as an argument and should return the fixed name.

In [12]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "dublin_ireland.osm"

# Keep track of unusual street names
# The value for each of the keys will be a set
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Court", "Place", "Square", "Lane", "Road", "Alley", "Arcade",
            "Bridge", "Brook", "Centre", "Close", "Crescent", "Green", "Grove", "Parade", "Park", "Terrace"]

mapping = { "Aveune": "Avenue",
            "Ave": "Avenue",
            "Cente": "Centre",
            "Cres":"Crescent",
            "hall":"Hall",
            "Heichts":"Heights",
            "lane":"Lane",
            "Rd":"Road",
            "Rd.":"Road",
            "road":"Road",
            "Roafd":"Road",
            "St":"Street",
            "St.":"Street",
            "street":"Street",
            "square":"Square",
            "heights":"Heights"
            }
#TODO: remove post code at end of street name?

# Find street names that aren't in the expected list and add to street_types dictionary
# Add specific street name to the set of values for a given street_type key
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

# Find out if nested tag is a street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    #pprint.pprint(dict(street_types))
    osm_file.close()
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m.group() not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    return name



## 1. Problems Encountered in the Map

1.

2.

3.

4.

## 2. Preparing Data for Database

After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so we will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files.  These csv files can then easily be imported to a SQL database as tables. 

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "dublin_ireland.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # TODO: transform each element into the correct format
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


## 4. Data Exploration

## 5. Additional ideas

## Conclusion

### Files